In [42]:
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from collections import Counter
import codecs
from sklearn.cluster import KMeans
from sklearn import preprocessing 
from scipy.sparse import coo_matrix
import numpy

In [43]:
# Abrir y leer el archivo
filename = "recursos/poesia_completa.txt"
text_file = open(filename, "r")
dataset = text_file.read()
dataset = dataset.lower() # Dejar el texto en minúsculas
text_file.close()

In [54]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    words = [token for token in tokens if token.isalpha()] # Quitar 'palabras' que no sean alfabéticas
    words = [token for token in words if token not in stopwords.words('spanish')] # Quitar sotpwords

    wnl = nltk.WordNetLemmatizer()
    lemmatized = [wnl.lemmatize(t) for t in words] # Convertir las palabras a su lemma
    
    return lemmatized

In [55]:
# Lista de tokens en base a su lemma
token_words = tokenize(dataset)

In [56]:
# Extraigo las palabras más frecuentes desde su lemma
frequent_words = []
counter = Counter(token_words)

# Dejo aquellas que tengan una frecuencia mayor a 10
for w in counter:
    if (counter[w]>=10):
        frequent_words.append(w)

In [62]:
WINDOWS_SIZE = 3

In [91]:
# Crear matriz de coocurrencia. Crear sólo columnas para las palabras que están en frequent_words
set_all_words={}
set_freq_words={}
data=[]
row=[]
col=[]
sents = nltk.sent_tokenize(dataset)
for sent in sents:
    tokens=tokenize(sent)
    for pos,token in enumerate(tokens):
        i=set_all_words.setdefault(token,len(set_all_words))
        start=max(0,pos-WINDOWS_SIZE)
        end=min(len(tokens),pos+WINDOWS_SIZE+1)
        for pos2 in range(start,end):
            if pos2==pos or tokens[pos2] not in frequent_words:
                continue
            j=set_freq_words.setdefault(tokens[pos2],len(set_freq_words))
            data.append(1.); row.append(i); col.append(j);
cooccurrence_matrix=coo_matrix((data,(row,col)))

In [92]:
# Generar clusters de palabras utilizando el algoritmo k-means.
def gen_clusters(vectors, clusters_num):
    vectors = preprocessing.normalize(vectors)
    km_model = KMeans(n_clusters=clusters_num)
    km_model.fit(vectors)

    return km_model

In [93]:
vocabulary = set_all_words
features=set_freq_words
model = gen_clusters(cooccurrence_matrix,35)

In [96]:
    # Show results
    c = Counter(sorted(model.labels_))
    print("\nTotal clusters:",len(c))
    for cluster in c:
        print ("Cluster#",cluster," - Total words:",c[cluster])

    # Show top terms and words per cluster
    print("\nTop terms and words per cluster:")
    print()
    #sort cluster centers by proximity to centroid
    order_centroids = model.cluster_centers_.argsort()[:, ::-1] 

    keysFeatures = list(features.keys())
    keysVocab = list(vocabulary.keys())
    for n in range(len(c)):
        print("#### Cluster %d #####" % n)
        print("Frequent terms:", end='')
       
        for ind in order_centroids[n, :10]:
            print(' %s' % keysFeatures[ind], end=',')

        print("\n")
        print("Words:", end='')
        word_indexs = [i for i,x in enumerate(list(model.labels_)) if x == n]
        for i in word_indexs:
            print(' %s' % keysVocab[i], end=',')
        print()
        print()

    print()


Total clusters: 35
Cluster# 0  - Total words: 45
Cluster# 1  - Total words: 65
Cluster# 2  - Total words: 80
Cluster# 3  - Total words: 79
Cluster# 4  - Total words: 1
Cluster# 5  - Total words: 112
Cluster# 6  - Total words: 2778
Cluster# 7  - Total words: 76
Cluster# 8  - Total words: 146
Cluster# 9  - Total words: 169
Cluster# 10  - Total words: 93
Cluster# 11  - Total words: 133
Cluster# 12  - Total words: 116
Cluster# 13  - Total words: 22
Cluster# 14  - Total words: 240
Cluster# 15  - Total words: 27
Cluster# 16  - Total words: 68
Cluster# 17  - Total words: 60
Cluster# 18  - Total words: 32
Cluster# 19  - Total words: 15
Cluster# 20  - Total words: 57
Cluster# 21  - Total words: 59
Cluster# 22  - Total words: 37
Cluster# 23  - Total words: 68
Cluster# 24  - Total words: 47
Cluster# 25  - Total words: 68
Cluster# 26  - Total words: 18
Cluster# 27  - Total words: 93
Cluster# 28  - Total words: 55
Cluster# 29  - Total words: 18
Cluster# 30  - Total words: 181
Cluster# 31  - Total 

Words: poesía, completa, tierra, ajena, egoísmo, adolescencia, aire, muriendo, rimbaud, días, querer, blancos, movible, vox, arqueada, aéreas, vivir, mil, nimias, cruzadas, cielo, hoja, actual, deshilacha, pelo, árboles, sitúan, vocablos, marcos, rozados, hueso, agitan, cocktail, humeante, calorías, desaparecen, repicante, austeridad, enredan, separados, pelos, moviendo, huellas, marciano, cognac, rasca, retretes, sanguíneos, fonean, pescar, calandria, eufórica, chapas, latosas, ascendente, faena, números, trillada, brillos, lloraban, rugidos, tiernas, sino, derredor, cosmos, volteretas, nenúfares, coagulan, frente, intemporal, brilla, cavidades, barrosas, tremendo, espesor, uñas, costura, desclavada, humor, repiqueo, rayada, cadáveres, llorosos, mar, salino, quitará, jabón, banderines, colorado, mano, derecha, comidas, nemo, llegará, lejos, día, raro, verdor, cantaré, vientos, oculta, húmeda, marcando, relojito, enviado, basuras, grandes, soledad, vertiente, olores, retina, brisas, dó

Words: amarillo, sol, contoneándose, repulsiva, traspasa, pieles, infierno, negro, calaveras, estrenadas, dejaba, revientan, exquisitas, delante, rojo, infantil, hermosos, balbuceante, colorido, impuro, inhibido, cobriza, niebla, alegrías, burbujean, intensas, armonías, equidistantes, luego, mañana, trakl, jaula, afuera, miran, cantan, pequeños, anduvieron, tarde, mañanas, ayer, contemporánea, colocado, rayos, animal, reconocer, partición, mesa, madre, ebriedad, miedosas, xi, refiero, sabes, vibra, humillado, mostraban, bergantín, moriría, representaba, debilísimo, filtrándose, heredado, filtrado, filtrada, caminaría, faltan, acopla, sobrevuela, dinastía, puesta, pondrán, volatinera, mentiras, ensombrece, espanto, primavera, desconocedora, milagros, cerró, iluminó, cerrarse, velada, reverbera, tortugas, gogh, miró, separó, leyenda, unas, éluard, pegar, elementales, rememoro,

#### Cluster 11 #####
Frequent terms: palabras, noche, cuerpo, lenguaje, muertos, lengua, silencio, dentro, poe

Words: bajo, soleado, rincón, sombra, tritura, maúlla, poemas, todavía, razón, genio, abstracto, única, gimiendo, efigie, amodorrada, aclara, siniestro, amar, nuevos, nombres, muerto, nueva, efecto, error, versiones, iluminada, galería, vaga, náufragos, je, cuarto, abierta, llamaban, ivonne, murió, densas, vigilas, temible, xix, evocar, bello, quieres, doble, sustrae, fuma, incluye, textos, aguardadora, insomne, abrazando, borró, nota, manuscrita, título, personaje, menciona, presentan, niñita, desconcertada, trabaja, asombrada, prefacio, empecemos, consocias, albacea, vistió, luto, terriblemente, afligida, social, clientes, atendía, ambo, hiciéronse, gabinete, alumbrándose, proyecta, régarde, k, lloró, habló, devant, est, louable, usaré, emerja, podemos, omitida, comienzo, estrofa, editada, máquina, luchado,

#### Cluster 28 #####
Frequent terms: tiempo, estrella, lugar, sueño, espacio, mismo, cuerpo, lejos, sé, silencio,

Words: reminiscencias, tiempo, estranguló, estrella, cuatro, i